# Notebook that runs model code and exports all necessary data packaged for plotting results in R

In [1]:
# first set parameters and modelsetup

In [1]:
import os
cwd = os.getcwd() 
print(cwd)

/Users/bpo/Documents/GitHub/BennyPhD/OSM2020_poster/03Model/phydra_OSM/notebooks


In [2]:
# allow import of my own local version of the phydra model (based on github folder structure)
import sys
sys.path.append('../../phydra_OSM/')

import pandas

# then import necessary packages and phydra code
import time
import numpy as np
from lmfit import Parameters
from phydra.model import cariaco
from phydra.core import ModelSetup

from scipy.integrate import odeint

In [3]:
# set up parameters
######### PARAMETER SETUP #############

parameters = Parameters()

# NUTRIENT(s)
parameters.add('nuts_num', value=2)
parameters.add('nuts1_nuttype', value=0)  # Nitrate
parameters.add('nuts2_nuttype', value=1)  # silicate

# PHYTOPLANKTON(s)
parameters.add('phyto_num', value=2)

parameters.add('OptI', value=40, vary=False)    # Optimum irradiance (einstein*m^-2*d^-1)
parameters.add('moP', value=0.1, vary=False)    # Phytoplankton mortality (d^-1)


#parameters.add('ratioSi', value=0, vary=False)  # Silicate ratio ## FALLBACK PARAM FOR OTHER PFTs
parameters.add('U_N', value=1.5, vary=False)    # Nitrate Half Saturation Constant
#parameters.add('U_P', value=0, vary=False)    # Phosphate Half Saturation Constant
parameters.add('U_Si', value=0, vary=False)   # Silicate Half Saturation Constant
parameters.add('muP', value=1.2, vary=False)    # Phytoplankton maximum growth rate (d^-1)
parameters.add('v', value=1, vary=False)    # Phytoplankton sinking rate (m d^-1)

# PFTs:
# DIATOMS
parameters.add('phyto1_muP', value=1.7, vary=False)
parameters.add('phyto1_U_N', value=1.5, vary=False)
parameters.add('phyto1_OptI', value=30, vary=False)
parameters.add('phyto1_v', value=1, vary=False)
parameters.add('phyto1_U_Si', value=1., vary=False)   # Silicate Half Saturation Constant

# HAPTO
parameters.add('phyto2_muP', value=1., vary=False)
parameters.add('phyto2_U_N', value=1., vary=False)
parameters.add('phyto2_OptI', value=45, vary=False)
parameters.add('phyto2_U_Si', value=0, vary=False)

# ZOOPLANKTON(s)
parameters.add('zoo_num', value=1)
parameters.add('moZ', value=0.01, vary=False)        # Zooplankton mortality (d^-1)
parameters.add('deltaZ', value=0.75, vary=False)    # Zooplankton Grazing assimilation coefficient (-)

parameters.add('Kp', value=1.5, vary=False)     # Zooplankton Grazing saturation constant (-)
parameters.add('pred', value=0.1, vary=False)  # quadratic higher order predation rate on zooplankton
parameters.add('muZ', value=1.0, vary=False)    # Zooplankton maximum grazing rate (d^-1)

# ZOO Feed Prefs
parameters.add('zoo1_P1', value=.35, vary=False)
parameters.add('zoo1_P2', value=.55, vary=False)

parameters.add('zoo1_D1', value=.1, vary=False)
# CONVERT FEEDPREFS TO GRAZEPREF FOR CALCULATION OF GRAZING

parameters.add('phyto1_Z1', value=parameters['zoo1_P1'].value, vary=False)
parameters.add('phyto2_Z1', value=parameters['zoo1_P2'].value, vary=False)

parameters.add('det1_Z1', value=parameters['zoo1_D1'].value, vary=False)

# DETRITUS (non-biotic pools)
parameters.add('det_num', value=1)
parameters.add('deltaD_N', value=0.01, vary=False)   # Nitrate Remineralization rate (d^-1)

# PHYSICS
#parameters.add('kappa', value=0.1, vary=False)  # vary=False) # min=0.09, max=0.11) # Diffusive mixing across thermocline (m*d^-1)
parameters.add('kw', value=0.04, vary=False)     # Light attenuation constant of water (m^-1)
parameters.add('kc', value=0.03, vary=False)      # Light attenuation via phytoplankton pigment (m^-1)

#NEW EMPOWER:
#parameters.add('moP_quad', value=0.025, vary=False)    # Phytoplankton mortality (d^-1)

parameters.add('wmix', value=0.1, vary=False)
parameters.add('beta_feed', value=0.69, vary=False)
parameters.add('kN_feed', value=0.75, vary=False)
parameters.add('vD', value=1., vary=False)

In [4]:
# now set up model

######### MODEL EVALUATION CODE #############

ms = ModelSetup(parameters, physics='Box', forcing='fullTS', time=None, pad=True, extendstart=True)
# TODO: TO get Extend Start to work properly, create mean forcing to add to start, and make sure transition to real forcing is smooth!

n, p, z, d = ms.classes

physx = ms.physics

N0 = 2.
P0 = 0.1
Z0 = 0.1
D0 = 0.1

initnut = [N0 for i in range(n.num)]
initphy = [P0 for i in range(p.num)]
initzoo = [Z0 for i in range(z.num)]
initdet = [D0 for i in range(d.num)]
initout = [0 for i in range(23)]
initcond = np.concatenate([initnut, initphy, initzoo, initdet,initout], axis=None)

#7737
timedays = np.arange(0., 8767., 1.0)

Nitrate 0 created
Silicate 1 created
phyto 0 created
phyto 1 created
D feedpref [0.1] P feedpref [0.35, 0.55]
zoo 0 created
det 0 created
x258depth forcing created
NO3_NO2_USF forcing created
SiO4_USF forcing created
Temperature forcing created
PAR forcing created
Verification data created


## run model!

In [5]:
# INTEGRATE:
tos = time.time()
print('starting integration')
outarray = odeint(cariaco, initcond, timedays, args=(ms, None))  # for some reason need to pass 2 args
tos1 = time.time()
print('finished after %4.3f sec' % (tos1 - tos))

#print(outarray)

starting integration
finished after 6.751 sec


In [6]:
# create aggTS model verification data for later analysis

ms1 = ModelSetup(parameters, physics='Box', forcing='aggTS', time='regime1')
ms2 = ModelSetup(parameters, physics='Box', forcing='aggTS', time='regime2')

Nitrate 0 created
Silicate 1 created
phyto 0 created
phyto 1 created
D feedpref [0.1] P feedpref [0.35, 0.55]
zoo 0 created
det 0 created
x258depth forcing created
NO3_NO2_USF forcing created
SiO4_USF forcing created
Temperature forcing created
PAR forcing created
Verification data created
Nitrate 0 created
Silicate 1 created
phyto 0 created
phyto 1 created
D feedpref [0.1] P feedpref [0.35, 0.55]
zoo 0 created
det 0 created
x258depth forcing created
NO3_NO2_USF forcing created
SiO4_USF forcing created
Temperature forcing created
PAR forcing created
Verification data created


# NOW collect and save output!

In [7]:
import matplotlib
#matplotlib.use('WxAgg')

import matplotlib.pyplot as plt

In [8]:
outarray_reg1, timedays_reg1, df_reg1 = ms.physics.forcing.NOX.returnModelOut_Regime(outarray,timedays, regime=1)
outarray_reg2, timedays_reg2, df_reg2 = ms.physics.forcing.NOX.returnModelOut_Regime(outarray,timedays, regime=2)

timedays_reg1x = np.mod(timedays_reg1-timedays_reg1[0], 365)
timedays_reg2x = np.mod(timedays_reg2-timedays_reg2[0], 365)

#if ms.physics.forcing.NOX.extendstart:
#    print(len(outarray), len(timedays))
#    outarray = ms.physics.forcing.NOX.returnModelOut_nospinup(outarray)
#    timedays = ms.physics.forcing.NOX.returnTimeDays_nospinup(timedays)

nn = ms.nutrients.num
pn = ms.phytoplankton.num
zn = ms.zooplankton.num
dn = ms.detritus.num
nuts = slice(0, nn)
phyto = slice(nn, nn+pn)
zoo = slice(nn + pn, nn + pn + zn)
det = slice(nn + pn + zn, nn + pn + zn + dn)
flux = slice(nn + pn + zn + dn, None)

outindex = nn + pn + zn + dn
print('outindex', outindex)

outindex 6


In [9]:
def returnmodoutdat(MS,OUTARRAY,timed, regime):
    N = OUTARRAY[:, nuts][:, 0]
    Si = OUTARRAY[:, nuts][:, 1]
    Pall = OUTARRAY[:, phyto]
    PFT_all = np.sum(Pall, axis=1)
    PFT_1 = Pall[:, 0]
    PFT_2 = Pall[:, 1]
    Zall = OUTARRAY[:, zoo][:,0]
    Det = OUTARRAY[:, det][:,0]
    
    if regime == '_reg1':
        df_reg = df_reg1
    else:
        df_reg = df_reg2
    
    
    data = {'N':N,
            'Si':Si,
            'PFT_all':PFT_all,
            'PFT_1':PFT_1,
            'PFT_2':PFT_2,
            'Zall':Zall,
            'Det':Det,
           'time':timed,
           'date':df_reg.index}
    return data

In [10]:
# intialise data of lists. 
modout_reg1 = returnmodoutdat(ms1,outarray_reg1, timedays_reg1,'_reg1')
# Create DataFrame 
out_1 = pandas.DataFrame(modout_reg1) 
# Print the output. 
out_1.to_csv('modelout_reg1.csv')

In [11]:
# intialise data of lists. 
modout_reg2 = returnmodoutdat(ms2, outarray_reg2, timedays_reg2,'_reg2')
# Create DataFrame 
out_2 = pandas.DataFrame(modout_reg2) 
# Print the output. 
out_2.to_csv('modelout_reg2.csv')

In [12]:
# as of yet not used!
#N = MS.physics.forcing.verif.fullpadNO2NO3
#Si = MS.physics.forcing.verif.fullpadSiOH
#chla = MS.physics.forcing.verif.fullpadFluorChla
#chla2 = MS.physics.forcing.verif.fullpadHPLC
#ZBM = MS.physics.forcing.verif.fullpadZoo
#PN = MS.physics.forcing.verif.fullpadPN

In [13]:
NOX = ms.physics.forcing.NOX.return_interpvalattime(timedays_reg1)

NOXdat = ms.physics.forcing.NOX.forcingfile

NN = ms.physics.forcing.NOX.rawforcing

SiOX = ms.physics.forcing.SiOH.return_interpvalattime(timedays_reg1)

SiOXdat = ms.physics.forcing.SiOH.forcingfile

SiX = ms.physics.forcing.SiOH.rawforcing

MLDRAW = ms.physics.forcing.X258.rawforcing

MLD = ms.physics.forcing.X258.return_interpvalattime(timedays_reg1)
# MLDderiv = ms.physics.forcing.X258.return_derivattime(timedays_ly)
MLDdat = ms.physics.forcing.X258.forcingfile
MLD_max = 150  # np.max(MLDdat) + 0.1 * np.max(MLDdat)

PARAW = ms.physics.forcing.PAR.rawforcing

PAR = ms.physics.forcing.PAR.return_interpvalattime(timedays_reg1)
PARdat = ms.physics.forcing.PAR.forcingfile

TmldRAW = ms.physics.forcing.SST.rawforcing

Tmld = ms.physics.forcing.SST.return_interpvalattime(timedays_reg1)
Tmlddat = ms.physics.forcing.SST.forcingfile

In [14]:
P_Max = 2  # np.max(Pall) + 0.9 * np.max(Pall)
N_Max = 20  # np.max(ms.physics.forcing.NOX.return_interpvalattime(timedays)) + np.max(ms.physics.forcing.NOX.return_interpvalattime(timedays)) * 0.1
D_Max = 1.5  # np.max(outarray_ly[:, 3]) + 0.2 * np.max(outarray_ly[:, 3])
Z_Max = 0.5  # np.max(Zall) + 0.1 * np.max(Zall)
P_Max = 2  # np.max(Pall) + 0.9 * np.max(Pall)

In [15]:
def returnregdat(timed,timex, regime):
    NOX = ms.physics.forcing.NOX.return_interpvalattime(timed)
    SiOX = ms.physics.forcing.SiOH.return_interpvalattime(timed)
    MLD = ms.physics.forcing.X258.return_interpvalattime(timed)
    PAR = ms.physics.forcing.PAR.return_interpvalattime(timed)
    Tmld = ms.physics.forcing.SST.return_interpvalattime(timed)
    
    if regime == '_reg1':
        df_reg = df_reg1
    else:
        df_reg = df_reg2
    
    
    
    data = {'NOX':NOX,
           'SiOX':SiOX,
           'x258':MLD,
           'PAR':PAR,
           'Tmld':Tmld,
           'time':timed,
           'yday':timex+1,
           'date':df_reg.index}
    return data

In [16]:
# intialise data of lists. 
forcing_reg1 = returnregdat(timedays_reg1, timedays_reg1x, '_reg1')
# Create DataFrame 
df_1 = pandas.DataFrame(forcing_reg1) 
# Print the output. 
df_1.to_csv('forcing_reg1.csv')

In [17]:
# intialise data of lists. 
forcing_reg2 = returnregdat(timedays_reg2, timedays_reg2x, '_reg2')
# Create DataFrame 
df_2 = pandas.DataFrame(forcing_reg2) 
# Print the output. 
df_2.to_csv('forcing_reg2.csv')

In [18]:
# get full output:

df_full_X = ms.physics.forcing.NOX.returnFullDF_nospinup()
outarray_X = ms.physics.forcing.NOX.returnModelOut_nospinup(outarray, full=True)
timedays_X = ms.physics.forcing.NOX.returnTimeDays_nospinup(timedays, full=True)

No need to remove spinup phase
No need to remove spinup phase


In [19]:
#df_full_X

In [20]:
TOToutarray = pandas.DataFrame(outarray_X, columns=['N','Si','P1','P2','Z','D',
                                      'PTempDepGrow','PNutUptake','PLightHarv','PGains',
                                      'Mix','','PMortality','PZooGrazed','PSinking','PLosses',
                                      'ZGains','ZLinMort','ZQuadMort','SiMixing','ZLosses',
                                      'ZUnassimFeedDetritus','DGains','DRemin','DZooGrazed',
                                      'DSinking', 'DLosses','NMixing','ZUnassimFeedNitrate'])

TOToutarray.insert(0, "time", timedays_X) 

TOToutarray.insert(0, "date", df_full_X.index) 

TOToutarray.to_csv('outarray_full.csv')

In [21]:
TOToutarray

,date,time,N,Si,P1,P2,Z,D,PTempDepGrow,PNutUptake,...,SiMixing,ZLosses,ZUnassimFeedDetritus,DGains,DRemin,DZooGrazed,DSinking,DLosses,NMixing,ZUnassimFeedNitrate
0,1993-01-15,0.0,2.000000,2.000000,0.100000,0.100000,0.100000,0.100000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1993-01-16,1.0,2.045163,2.031145,0.102358,0.100097,0.098260,0.119008,1.818703,2.100578,...,0.003974,0.000277,0.042388,0.002095,0.000097,0.002095,0.004287,0.138748,0.000154,0.004012
2,1993-01-17,2.0,2.088323,2.060728,0.104964,0.100283,0.096580,0.137870,2.727113,3.163741,...,0.005897,0.000423,0.063891,0.003380,0.000168,0.003380,0.006927,0.206155,0.000235,0.006050
3,1993-01-18,3.0,2.129489,2.088752,0.107819,0.100549,0.094961,0.156622,3.634783,4.236424,...,0.007772,0.000576,0.085680,0.004852,0.000260,0.004852,0.009964,0.272032,0.000321,0.008118
4,1993-01-19,4.0,2.168671,2.115216,0.110921,0.100887,0.093402,0.175298,4.541725,5.317980,...,0.009600,0.000738,0.107789,0.006512,0.000375,0.006512,0.013399,0.336419,0.000411,0.010219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8762,2017-01-11,8762.0,3.725664,3.309753,0.082992,0.232378,0.088089,0.933437,8075.598434,8616.565974,...,61.020592,36.545202,282.553463,91.323382,99.054170,91.323382,281.700933,160.757219,20.335637,22.881198
8763,2017-01-12,8763.0,3.779917,3.348904,0.089029,0.234900,0.088756,0.945599,8076.512013,8617.904986,...,61.022258,36.546600,282.587707,91.332777,99.057461,91.332777,281.723015,160.846234,20.336414,22.884394
8764,2017-01-13,8764.0,3.833777,3.387331,0.095510,0.237350,0.089484,0.958333,8077.423877,8619.250074,...,61.023944,36.548046,282.622880,91.342296,99.060862,91.342296,281.745454,160.936083,20.337219,22.887677
8765,2017-01-14,8765.0,3.887178,3.424952,0.102460,0.239717,0.090278,0.971660,8078.334032,8620.601061,...,61.025650,36.549543,282.659025,91.351945,99.064382,91.351945,281.768272,161.026755,20.338052,22.891052
